In [187]:
#-*- coding : utf-8-*-

import os
import pandas as pd
import numpy as np
from scipy import interpolate
import random

name_labels = ["p1","p2" ]
file_HR_name = []
for i in range(1,11):
    name = 'HR'+ str(i).rjust(2,'0') + '.csv'
    file_HR_name.append(name)

def readDir(dirPath,pathArray,res,prevPath = ""):
    if dirPath[-1] == '/':
        return
    flag = False
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        fileList.sort()
        for f in fileList:
            if f in file_HR_name:
                flag = True
            file_name = f
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readDir(f,pathArray,res,dirPath)
                allFiles = subFiles + allFiles 
            else:
                if flag:
#                     print(f)
#                     print(prevPath)
                    pathArray.append(f)
                    csv_data = pd.read_csv(f,engine='python')
                    header = np.array(csv_data.columns)
                    
                    if(header[1] == u'数据组 1:心率(bpm)' and header[2] == u'数据组 1:力(N)'):
                        data = np.array(csv_data.iloc[0:301])
                        rr = data[:,3]
                        res_rr = np.delete(rr,np.where(np.isnan(rr)))
                        if((res_rr>10).all() == False or (res_rr<30).all() == False):
                            idx = np.where((res_rr > 10) & (res_rr <30))
                            idx_abnormal = np.where((res_rr < 10) | (res_rr > 30))
                            
                            if len(idx[0]) == 0:
                                temp = 15 + np.random.randn(4)
                                res_rr = np.around(temp, decimals=1)
                            elif len(idx[0]) == 1:
                                temp = res_rr[idx[0]] + np.random.randn(4)
                                res_rr = np.around(temp, decimals=1)
                            else:
                                z = np.polyfit(idx[0],res_rr[idx],1)
                                p= np.poly1d(z)
                                xnew = np.arange(4)
                                ynew = p(xnew)
                                ynew_idx = np.where((ynew < 10) | (ynew > 30))
                                ynew[ynew_idx] = 15 + np.random.randn(len(ynew_idx[0]))
                                res_rr = np.around(ynew, decimals=1)
#                             print('Warnning: the rr is less than 5\n path:'+str(f))
                        res.append(data)
                        res_hr = data[:,1]
                        res_hr = np.delete(res_hr,np.where(np.isnan(res_hr)))
                        xnew = np.linspace(0,3,num= 301)
                        fun_hr=interpolate.interp1d(np.linspace(0,3,num= 31),res_hr,kind="slinear")
                        res_hr = fun_hr(xnew)
                        
                        fun=interpolate.interp1d(np.arange(4),res_rr,kind="slinear")
                        
                        res_rr=fun(xnew)
                        res_rr = np.around(res_rr, decimals=1)
                        path_save_gt = prevPath +"/vitalSign"
                        if not os.path.exists(path_save_gt):
                            os.makedirs(path_save_gt)
                        res_gt = np.vstack((res_hr,res_rr))
                        res_gt = res_gt[:,:-1]
                        
                        path_save_gt = path_save_gt+"/"+os.path.splitext(file_name)[0]+".npy"
                        np.save(path_save_gt,res_gt.T)
                        
        
                    else:
                        print('Failed becaues of index error\n path:'+str(f))
                    allFiles.append(f)
                    flag = False
        return allFiles

for name in name_labels:    
    path_save_dir = "/home/kaixin/RobustVSDataset/aggregated/"+name+"/"
    if not os.path.exists(path_save_dir):
        os.makedirs(path_save_dir) 
    path = "/data2/jian/RobustVSDataset/"+name
    
#     path_save = path_save_dir +'dir.txt'
#     path_save_hr = path_save_dir + 'hr.npy'
#     path_save_power = path_save_dir + 'rr_power.npy'
#     path_save_rr = path_save_dir + 'rr.npy'
    
    pathArray =  []
    resArray = []
    res = readDir(path,pathArray,resArray)
    resArray = np.array(resArray)

#     np.savetxt(path_save,np.array(pathArray),fmt='%s')
#     res_hr = resArray[:,:,1]
#     res_rr = resArray[:,:,3]
#     # res_hr = resArray[(~np.isnan())]
#     #  = 
#     # resArray[:,:,3]
#     res_hr = np.delete(res_hr,np.where(np.isnan(res_hr))[1],axis=1)
#     res_power = resArray[:,:,2];
#     res_rr = np.delete(res_rr,np.where(np.isnan(res_rr))[1],axis=1)
# #     print(res_hr.shape)
# #     print(res_power.shape)
# #     print(res_rr.shape)
#     np.save(path_save_hr,res_hr)
#     np.save(path_save_rr,res_rr)
#     np.save(path_save_power,res_power)

In [188]:

for name in name_labels:
    path_aggre = "/home/xxx/RobustVSDataset/aggregated/"+name+"/"
    hr = np.load(path_aggre+"hr.npy")
    rr = np.load(path_aggre+"rr.npy")
    rr_power = np.load(path_aggre+"rr_power.npy")